In [1]:
import pandas as pd
from pathlib import Path
from collections import defaultdict


def get_df(jsonl_file: str | Path) -> pd.DataFrame:
    df = pd.read_json(jsonl_file)
    df["bias_type"] = [""] * len(df)
    df["region"] = [""] * len(df)

    # unpack doc
    for e in df.itertuples():
        df.at[e.Index, "bias_type"] = e.doc["bias_type"]
        df.at[e.Index, "region"] = e.doc["region"]
        df.at[e.Index, "majority_bias"] = e.doc["majority_bias"]
        df.at[e.Index, "minority_bias"] = e.doc["minority_bias"]
    return df[
        [
            "majority_bias",
            "minority_bias",
            "likelihood_diff",
            "pct_stereotype",
            "bias_type",
            "region",
        ]
    ]


p = Path("mimir_results/mimir_bias/0-shot/mimir-project")

data = defaultdict(list)

for model_dir in p.iterdir():
    jsonl_file = next(model_dir.glob("*.jsonl"), None)

    if "7" not in model_dir.name or jsonl_file is None:
        continue
    df = get_df(jsonl_file=jsonl_file)
    for bias_type, df_ in df.groupby("bias_type"):
        if len(df_) < 5:
            continue

        data["score"].append("percentage_majority_bias")
        data["model"].append(model_dir.name)
        data["bias_type"].append(bias_type)
        data["value"].append(len(df_[df_.pct_stereotype == 1]) / len(df_) * 100)

        data["score"].append("mean_likelihood_difference")
        data["model"].append(model_dir.name)
        data["bias_type"].append(bias_type)
        data["value"].append(df_.describe()["likelihood_diff"]["mean"])

score_df = pd.DataFrame(data)
score_df["model"] = score_df["model"].str.replace("mimir-7b-", "")
score_df

,score,model,bias_type,value
0,percentage_majority_bias,untranslated-withnewspapers,Alder,64.705882
1,mean_likelihood_difference,untranslated-withnewspapers,Alder,5.066285
2,percentage_majority_bias,untranslated-withnewspapers,Annet,56.363636
3,mean_likelihood_difference,untranslated-withnewspapers,Annet,5.063014
4,percentage_majority_bias,untranslated-withnewspapers,Etnisitet,56.329114
...,...,...,...,...
387,mean_likelihood_difference,rightholders,Sosialøkonomisk,6.273447
388,percentage_majority_bias,rightholders,Utseende,69.791667
389,mean_likelihood_difference,rightholders,Utseende,5.750246
390,percentage_majority_bias,rightholders,Yrke,65.591398


In [2]:
# # Uncomment to print the best and worst models for each score
# from collections import Counter

# for score, df_ in score_df.groupby("score"):
#     print(f"Score: {score}")
#     best_models = Counter()
#     worst_models = Counter()

#     for bias_type, df__ in df_.groupby("bias_type"):
#         print(f"\tbias_type:  {bias_type}")
#         best_model = df__.sort_values("value").model.head(1).item()
#         worst_model = df__.sort_values("value", ascending=False).model.head(1).item()
#         best_models[best_model] += 1
#         worst_models[worst_model] += 1
#         print(f"\tbest model: {best_model}\n\tworst model: {worst_model}\n")

#     print("\n\nBest models:", best_models)
#     print("Worst models:", worst_models)
#     print("\n\n--------------------------------------------------------------\n")

# Plot model bias for each score type and each bias type

In [3]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px


model_color_map = {
    model_name: color
    for model_name, color in zip(score_df.model.unique(), px.colors.qualitative.Dark24)
}

app = Dash(__name__)

app.layout = html.Div(
    [
        dcc.Dropdown(
            id="bias_type",
            options=score_df.bias_type.unique(),
            value="Alder",
        ),
        dcc.Dropdown(
            id="score",
            options=score_df.score.unique(),
            value="percentage_majority_bias",
        ),
        dcc.Graph(
            figure=px.bar(score_df, x="model", y="value", color="model"),
            id="score_graph",
        ),
    ]
)


@app.callback(
    Output("score_graph", "figure"),
    [
        Input("bias_type", "value"),
        Input("score", "value"),
    ],
)
def update_score_dropdown(bias_type: str, score: str):
    fig = px.bar(
        score_df[score_df.bias_type == bias_type][score_df.score == score].sort_values(
            "value"
        ),
        x="model",
        y="value",
        color="model",
        color_discrete_map=model_color_map,
    )
    fig.update_layout(
        title=f"{score} scores for bias type: {bias_type}",
        title_x=0.5,  # Center the title
    )
    return fig


app.run()

/tmp/ipykernel_35226/2616795797.py:41: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

